# Preprocess the Text

In [1]:
# retrieve the text
def get_docs():
    from keras.utils import get_file
    print('\nFetching the text...')
    url = 'https://raw.githubusercontent.com/maxim5/stanford-tensorflow-tutorials/master/data/arxiv_abstracts.txt'
    path = get_file('arxiv_abstracts.txt', origin=url)
    with open(path) as file_:
        return file_.readlines()

In [2]:
# preprocess the text
def preprocess(string):
    import nltk
    from nltk.tokenize import sent_tokenize
    return [[word.lower() for word in nltk.word_tokenize(line) if word.isalpha()] 
            for line in sent_tokenize(string)]

In [3]:
sentences = []
for doc in get_docs():
    sentences.extend(preprocess(string=doc))
print("Retrieved " + str(len(sentences)) + " sentences.")

Using TensorFlow backend.



Fetching the text...
Retrieved 48168 sentences.


# Convert Text to Input

In [4]:
def flatten(orig):
    import numpy as np
    out = []
    for x in orig:
        out.extend(x)
    return out
    
# 2 level tree parallel reduce
def flatten_parallel(orig, n_threads=100):
    from multiprocessing import Pool
    from multiprocessing.dummy import Pool as ThreadPool
    p = ThreadPool(n_threads)
    length = len(orig)
    while(length > n_threads):
        step = length//n_threads
        step = 2 if step < 2 else step
        orig = [orig[x: min(x + step, length)] 
                 for x in range(0, length, step)]
        orig = p.map(flatten, orig)
        length = len(orig)
    p.close()
    return flatten(orig)

In [5]:
def text_to_input(sentences, method='last_word', overlap=10, overlap_len=40):
    if method == 'last_word':
        x = [s[:-1] for s in sentences]
        y = [s[-1] for s in sentences]
    elif method == 'n_overlap':
        all_words = flatten_parallel(sentences)
        overlapped = [all_words[x: min(x+overlap_len, len(all_words))] 
                                for x in range(0, len(all_words), overlap)]
        x = [s[:-1] for s in overlapped]
        y = [s[-1] for s in overlapped]
        sentences = overlapped
    return x, y, sentences

In [6]:
x, y, sentences = text_to_input(sentences, method='n_overlap')

# Embed Words as Vectors

In [7]:
def embed_words(sentences, debug=True):
    import gensim
    if debug:
        print("Creating embeddings...")
    embed_model = gensim.models.Word2Vec(
        sentences,
        size=100, # vector dimension
        min_count=1, # min num times it needs to be in sentences to count
        window=5, # num words around word that affect vector
        iter=100)
    if debug:
        print("Embedding model created.")
    return embed_model

def get_embedding_layer(embed_model):
    from keras.layers.embeddings import Embedding
    weights = embed_model.wv.vectors
    vocab_size, embedding_size = weights.shape
    return Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[weights])

def word2index(embed_model, word):
    return embed_model.wv.vocab[word].index

def index2word(embed_model, index):
    return embed_model.wv.index2word[index]

# Construct Model

In [10]:
def model(embed_model):
    from keras.layers.recurrent import LSTM
    from keras.layers.embeddings import Embedding
    from keras.layers import Dense, Activation
    from keras.models import Sequential
    
    vocab_size, embedding_size = embed_model.wv.vectors.shape
    model = Sequential()
    model.add(get_embedding_layer(embed_model))
    model.add(LSTM(units=embedding_size, input_shape=(None,)))
    model.add(Dense(units=vocab_size))
    model.add(Activation('softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    
    return model

# Fit and Predict Words

In [11]:
def fit(model, x_train, y_train):
    model.fit(x_train, y_train, 
             batch_size=8192,
             epochs=500,
             verbose=1)
    return model
    
def predict(model, x_test):
    print(model.predict(x_test))

In [12]:
embed_model = embed_words(sentences)

Creating embeddings...
Embedding model created.


In [13]:
import numpy as np
x = [np.array([word2index(embed_model, w) for w in words]) for words in x]
y = np.array([word2index(embed_model, w) for w in y])

In [14]:
# remove arrays not of size 39, temp fix
x = np.stack([a for a in x if a.shape[0] == 39], axis=0)

In [ ]:
import numpy as np
model = fit(model(embed_model), x[:100000], y[:100000])#y[:-4])

Epoch 1/500
100000/100000 [==============================] - 59s 591us/step - loss: 7.7016
Epoch 2/500
100000/100000 [==============================] - 56s 556us/step - loss: 7.3732
Epoch 3/500
100000/100000 [==============================] - 54s 541us/step - loss: 6.8077
Epoch 4/500
100000/100000 [==============================] - 56s 559us/step - loss: 6.1504
Epoch 5/500
100000/100000 [==============================] - 55s 548us/step - loss: 5.7051
Epoch 6/500
100000/100000 [==============================] - 54s 542us/step - loss: 5.3722
Epoch 7/500
100000/100000 [==============================] - 55s 552us/step - loss: 5.0678
Epoch 8/500
100000/100000 [==============================] - 57s 566us/step - loss: 4.7803
Epoch 9/500
100000/100000 [==============================] - 54s 544us/step - loss: 4.5057
Epoch 10/500
100000/100000 [==============================] - 62s 624us/step - loss: 4.2399
Epoch 11/500
100000/100000 [==============================] - 57s 568us/step - loss: 3.98

100000/100000 [==============================] - 59s 587us/step - loss: 0.0473
Epoch 91/500
100000/100000 [==============================] - 62s 619us/step - loss: 0.0461
Epoch 92/500
100000/100000 [==============================] - 63s 631us/step - loss: 0.0449
Epoch 93/500
100000/100000 [==============================] - 60s 604us/step - loss: 0.0437
Epoch 94/500
100000/100000 [==============================] - 58s 577us/step - loss: 0.0426
Epoch 95/500
100000/100000 [==============================] - 61s 608us/step - loss: 0.0415
Epoch 96/500
100000/100000 [==============================] - 59s 586us/step - loss: 0.0406
Epoch 97/500
100000/100000 [==============================] - 62s 616us/step - loss: 0.0396
Epoch 98/500
100000/100000 [==============================] - 59s 594us/step - loss: 0.0386
Epoch 99/500
100000/100000 [==============================] - 61s 611us/step - loss: 0.0377
Epoch 100/500
100000/100000 [==============================] - 61s 614us/step - loss: 0.0369
